In [52]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import matplotlib
import csv
matplotlib.rcParams['figure.figsize'] = (20, 10)

## Change the path to the extracted recording

In [53]:
path = "../1811calib"

In [54]:
pathL = os.path.join(path, '1')
pathR = os.path.join(path, '2')

In [82]:
CH_W = 7
CH_H = 5
EVERY_NTH_FRAME = 15
# Square size in mm 
SQUARE_SIZE = 30
# Termination criteria for refining the detected corners (taken from openCV docs as reasonable params)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


objp = np.zeros((CH_H*CH_W,3), np.float32)
objp[:,:2] = np.mgrid[0:CH_H,0:CH_W].T.reshape(-1,2)
objp *= SQUARE_SIZE

In [83]:

img_ptsL = []
img_ptsR = []
obj_pts = []
counter = 0

with open(os.path.join(path, 'match.csv')) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        counter += 1
        if counter % EVERY_NTH_FRAME == 0:
            print(f'{pathL}{row[1]}.png')
            imgL = cv2.imread(os.path.join(pathL, f'{row[2]}.png'))
            imgR = cv2.imread(os.path.join(pathR, f'{row[1]}.png'))
            print(f'Time difference: {int(row[2]) - int(row[1])} ns')
            imgL_gray = cv2.imread(os.path.join(pathL, f'{row[2]}.png'),0)
            imgR_gray = cv2.imread(os.path.join(pathR, f'{row[1]}.png'),0)

            outputL = imgL.copy()
            outputR = imgR.copy()

            retR, cornersR =  cv2.findChessboardCorners(outputR,(CH_H,CH_W),None)
            retL, cornersL = cv2.findChessboardCorners(outputL,(CH_H,CH_W),None)

            if retR and retL:
                obj_pts.append(objp)
                cv2.cornerSubPix(imgR_gray,cornersR,(11,11),(-1,-1),criteria)
                cv2.cornerSubPix(imgL_gray,cornersL,(11,11),(-1,-1),criteria)

                print(f'{cornersL[0]} - {cornersR[0]}')

                anchor_is_bottom = (cornersL[0][0,1] - cornersL[-1][0,1]) > 0
                is_bottom = (cornersR[0][0,1] - cornersR[-1][0,1]) > 0

#                 if anchor_is_bottom != is_bottom:
#                     print("MISMATCH. HUMAN IS DEAD")
#                     cornersR = np.flip(cornersR, 0)
                    
#                     cv2.drawChessboardCorners(outputR,(CH_H,CH_W),cornersR,retR)
#                     plt.subplot(1,2,1)
#                     plt.imshow(outputR)
#                     cv2.drawChessboardCorners(outputL,(CH_H,CH_W),cornersL,retL)
#                     plt.subplot(1,2,2)

#                     plt.imshow(outputL)
#                     plt.show()


                
                img_ptsL.append(cornersL)
                img_ptsR.append(cornersR)


# Calibrating left camera
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(obj_pts,img_ptsL,imgL_gray.shape[::-1],None,None)

hL,wL= imgL_gray.shape[:2]
new_mtxL, roiL= cv2.getOptimalNewCameraMatrix(mtxL,distL,(wL,hL),1,(wL,hL))

# Calibrating right camera
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(obj_pts,img_ptsR,imgR_gray.shape[::-1],None,None)

hR,wR= imgR_gray.shape[:2]
new_mtxR, roiR= cv2.getOptimalNewCameraMatrix(mtxR,distR,(wR,hR),1,(wR,hR))

../1811calib/11637256584007169618.png
Time difference: -39134 ns
[[544.7923  146.46465]] - [[384.3026  141.17389]]
../1811calib/11637256584507102926.png
Time difference: -52020 ns
[[523.6614  114.37659]] - [[358.8908 108.1522]]
../1811calib/11637256585007011310.png
Time difference: -42481 ns
[[542.6536   85.25384]] - [[372.30295  77.70494]]
../1811calib/11637256585506928694.png
Time difference: -36519 ns
[[534.5248  105.83635]] - [[ 69.28311 329.51114]]
../1811calib/11637256586006848232.png
Time difference: -37865 ns
../1811calib/11637256586506768040.png
Time difference: -38827 ns
../1811calib/11637256587006701539.png
Time difference: -45250 ns
../1811calib/11637256587506609346.png
Time difference: -40980 ns
../1811calib/11637256588006528731.png
Time difference: -40135 ns
../1811calib/11637256588506446653.png
Time difference: -38865 ns
../1811calib/11637256589006367730.png
Time difference: -38865 ns
[[247.28334 351.80627]] - [[ 33.640167 354.6008  ]]
../1811calib/11637256589506285845.p

[[502.42914 176.21823]] - [[380.53302 172.70004]]
../1811calib/11637256628000116818.png
Time difference: -39674 ns
[[441.87326 131.15869]] - [[315.76382 126.97111]]
../1811calib/11637256628500038240.png
Time difference: -41596 ns
../1811calib/11637256628999957509.png
Time difference: -40173 ns
../1811calib/11637256629499875316.png
Time difference: -38288 ns
../1811calib/11637256629999798470.png
Time difference: -42519 ns
../1811calib/11637256630499715777.png
Time difference: -38827 ns
../1811calib/11637256630999636046.png
Time difference: -42442 ns
[[197.85847 356.96457]] - [[ 48.98313 359.5088 ]]
../1811calib/11637256631499556584.png
Time difference: -39442 ns
[[213.47249 390.45447]] - [[ 64.89695 393.57547]]
../1811calib/11637256631999475930.png
Time difference: -36635 ns
[[218.24113 399.50732]] - [[ 72.40075 402.59952]]
../1811calib/11637256632499393314.png
Time difference: -38288 ns
[[226.43214 375.70966]] - [[340.22476 215.60062]]
../1811calib/11637256632999314622.png
Time differe

In [90]:
np.savez("intrinsic_data_left", mtx=mtxL, dist=distL, rvecs=rvecsL, tvecs=tvecsL)
np.savez("intrinsic_data_right", mtx=mtxR, dist=distR, rvecs=rvecsR, tvecs=tvecsR)

# Stereo calibration

In [91]:
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(obj_pts, img_ptsL, img_ptsR, new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], criteria_stereo, flags)

In [92]:
rectify_scale=1
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR= cv2.stereoRectify(new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], Rot, Trns, rectify_scale,(0,0))

In [93]:
np.savez("extrinsic_data", Q=Q)

## stereoCalibrate retval

According to the docs, represents the final value of the re-projection error.


In [94]:
retS

22.070372485940943

In [95]:
Trns

array([[-220.81985376],
       [  -0.42033527],
       [  46.11627142]])

 - **this translation value is off by a few mm from real-world situation (measured by a ruler), although the returned reproj error is not that big?**
 
 Should be something like: (-98, ~0, ~8)

In [96]:
Q

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -3.33977163e+02],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -2.75406767e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.83956436e+02],
       [ 0.00000000e+00,  0.00000000e+00,  4.43293180e-03,
        -0.00000000e+00]])

In [97]:
Left_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxL, distL, rect_l, proj_mat_l,
                                             imgL_gray.shape[::-1], cv2.CV_16SC2)
Right_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxR, distR, rect_r, proj_mat_r,
                                              imgR_gray.shape[::-1], cv2.CV_16SC2)

print("Saving parameters ......")
cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("Left_Stereo_Map_x",Left_Stereo_Map[0])
cv_file.write("Left_Stereo_Map_y",Left_Stereo_Map[1])
cv_file.write("Right_Stereo_Map_x",Right_Stereo_Map[0])
cv_file.write("Right_Stereo_Map_y",Right_Stereo_Map[1])
cv_file.write("Trns",Trns)
cv_file.release()

Saving parameters ......
